# Problem 20
You are given an M by N matrix consisting of booleans that represents a board. Each True boolean represents a wall. Each False boolean represents a tile you can walk on.

Given this matrix, a start coordinate, and an end coordinate, return the minimum number of steps required to reach the end coordinate from the start. If there is no possible path, then return null. You can move up, left, down, and right. You cannot move through walls. You cannot wrap around the edges of the board.

For example, given the following board:

```
[[f, f, f, f],
[t, t, f, t],
[f, f, f, f],
[f, f, f, f]]
```
and `start = (3, 0)` (bottom left) and `end = (0, 0)` (top left), the minimum number of steps required to reach the end is 7, since we would need to go through `(1, 2)` because there is a wall everywhere else on the second row.

---
## Test Cases

In [290]:
# test cases
test_1 = [['f', 'f', 'f', 'f'],
          ['t', 't', 'f', 't'],
          ['f', 'f', 'f', 'f'],
          ['f', 'f', 'f', 'f']]

test_2 = [['f', 'f', 'f', 'f'],
          ['t', 't', 't', 't'],
          ['f', 'f', 'f', 'f'],
          ['f', 'f', 'f', 'f']]

test_3 = [['f', 'f', 'f', 'f'],
          ['t', 'f', 'f', 't'],
          ['t', 't', 'f', 'f'],
          ['f', 't', 'f', 'f']]

test_4 = [['f', 'f', 'f', 'f'],
          ['t', 't', 'f', 't'],
          ['f', 'f', 'f', 'f'],
          ['f', 'f', 'f', 'f'],
          ['f', 'f', 'f', 'f'],
          ['t', 't', 'f', 't'],
          ['f', 'f', 'f', 'f'],
          ['f', 'f', 'f', 'f']]

test_5 = [['f', 'f', 'f', 'f', 'f', 'f', 't'],
          ['t', 't', 't', 'f', 'f', 't', 'f'],
          ['t', 'f', 'f', 'f', 't', 'f', 'f'],
          ['f', 'f', 't', 't', 'f', 't', 'f'],
          ['f', 't', 'f', 't', 'f', 'f', 'f'],
          ['f', 't', 'f', 'f', 't', 'f', 't'],
          ['f', 't', 't', 'f', 't', 'f', 'f'],
          ['f', 'f', 'f', 'f', 'f', 'f', 'f']]

def test_to_matrix(test):
    matrix = []
    for l in test:
        mini = []
        for e in l:
            if(e == "f"): mini.append(False)
            else: mini.append(True)
        matrix.append(mini)
    return matrix

test_1 = test_to_matrix(test_1)
test_2 = test_to_matrix(test_2)
test_3 = test_to_matrix(test_3)
test_4 = test_to_matrix(test_4)
test_5 = test_to_matrix(test_5)

---
## Solution

In [291]:
# solution code
import math

def wall_check(matrix):
    horizantal_walls = []
    vertical_walls = []
    # Check rows
    for row in matrix:
        if all(x == True for x in row):
            horizantal_walls.append(matrix.index(row))
    # Check columns
    for i in range(len(matrix[0])):
        if all(matrix[j][i] == True for j in range(len(matrix))):
            vertical_walls.append(i)
    return [horizantal_walls, vertical_walls]


def movement_check(matrix, position):
    # surroundings = above, below, left, right
    surroundings = {(position[0]-1, position[1]), (position[0]+1, position[1]), (position[0], position[1]-1), (position[0], position[1]+1)}
    remove_set = set()
    for tile in surroundings:
        if tile[0] < 0 or tile[1] < 0 or tile[0] >= len(matrix) or tile[1] >= len(matrix[0]) or matrix[tile[0]][tile[1]]:
            remove_set.add(tile)
    surroundings -= remove_set
    return list(surroundings)


def closest_to_endpoint(surroundings, endpoint, previous_points, matrix, exile_points = []):
    distances = []
    for tile in surroundings:
        if(tile not in exile_points):
            edist = math.dist([tile[0], tile[1]],[endpoint[0], endpoint[1]])
            distances.append([tile, edist])
    distances.sort(key = lambda x : x[1])
    result = None
    for tile in distances:
        if(tile[0] not in previous_points):
            result = tile[0]
            break
    if(result == None):
        exile_points = [previous_points[-1]]
        previous_points = previous_points[:-1]
        result, previous_points, exile_points = closest_to_endpoint(movement_check(matrix, previous_points[-1]), endpoint, previous_points, matrix, exile_points = exile_points)
    return result, previous_points, exile_points    


def min_steps(matrix, start, end):
    null_output = "No path found."
    if(matrix[end[0]][end[1]] == True or matrix[start[0]][start[1]] == True): return null_output
    movement_positions = movement_check(matrix, start)
    # edge case where no movement for starting position is possible
    if(movement_positions == []): return null_output

    # edge cases where start and end position blocked by wall
    walls = wall_check(matrix)
    for h in walls[0]:
        if(start[0] < h <end[0] or start[0] > h > end[0]):
            return null_output
    for v in walls[1]:
        if(start[1] < v < end[1] or start[1] > v > end[1]):
            return null_output

    # find path from start to end
    position = start
    previously_visited = [position]
    exiles =[]
    while(position != end):
        if(previously_visited[-1] == None):
            return null_output
        movement_positions = movement_check(matrix, position)
        
        # find next position based on surroundings closest euclidean distance to end point
        position, previously_visited, exiles  = closest_to_endpoint(movement_positions, end, previously_visited, matrix, exile_points = exiles)
        previously_visited.append(position)

    # uncomment to veiw path
    # print(previously_visited)
    
    return len(previously_visited) - 1


---
## Test Solution

In [292]:
# solution testing test cases
start = (3, 0)
end = (0, 0)
min_steps(test_1, start, end)

7

In [293]:
start = (3, 0)
end = (0, 0)
min_steps(test_2, start, end)

'No path found.'

In [294]:
start = (3, 0)
end = (0, 0)
min_steps(test_3, start, end)

'No path found.'

In [295]:
start = (6, 0)
end = (0, 0)
min_steps(test_4, start, end)

10

In [296]:
start = (0, 5)
end = (1, 6)
min_steps(test_5, start, end)

24

---
## Solution Explained

### Solution
This code provides a solution to a problem where you are given a boolean matrix (2D list) representing a board with walls and empty spaces. Given a start and an end coordinate, the function returns the minimum number of steps required to reach the end from the start, while avoiding the walls.

#### **wall_check(matrix)** <br>
This function takes in the board matrix and returns two lists: `horizantal_walls` and `vertical_walls` representing the indexes of all rows and columns that have all `True` values (walls). The function starts by iterating over each row of the matrix, using the `all()` function to check if all the values in the row are `True`. If this is the case, the function appends the index of the row to `horizantal_walls`. The same process is then repeated for each column to generate `vertical_walls`. The two lists are returned in a list.

The time complexity of this function is O(nm), where n is the number of rows and m is the number of columns in the matrix. The space complexity is O(n + m), as two lists are returned of length n and m.


#### **movement_check(matrix, position)**
This function takes in the board matrix and a coordinate `position` and returns a list of surrounding empty spaces (tiles) that can be reached from the current position. The function creates a set of all surrounding coordinates, then iterates over each coordinate to determine if it falls outside the bounds of the board or if it is a wall (True value in the matrix). Any coordinates outside the board or on a wall are removed from the set of surroundings. The function then returns the remaining surrounding tiles in list format.

The time complexity of this function is O(1), since it only checks the surrounding coordinates (four maximum) of the given position, and the loop that checks for walls or boundaries has a constant number of iterations. The space complexity is O(1), since the function only creates and modifies a set and a list.


#### **closest_to_endpoint(surroundings, endpoint, previous_points, matrix, exile_points = [])**
This function takes in the list of surrounding empty spaces, an endpoint coordinate, a list of previously visited coordinates `previous_points`, the board matrix, and an optional list of `exile_points` representing previously visited coordinates to avoid. The function calculates the Euclidean distance from each surrounding tile to the endpoint and stores these distances and their corresponding tiles in a list, which is sorted in ascending order based on the distances. The function then iterates over the sorted list, selecting the first tile that has not been previously visited. If such a tile exists, it is returned along with the updated `previous_points` and `exile_points`. If no such tile exists, the last tile in `previous_points` is added to `exile_points`, and the function is recursively called with the list of empty tiles from the current position, the endpoint, the list of previous points, and the updated `exile_points`. If this recursive call also returns no valid next tile, the function returns `None` along with the `previous_points` and `exile_points`.

The time complexity of this function is O(n log n), where n is the number of surrounding empty spaces. The math.dist() function used to calculate the Euclidean distances has a time complexity of O(n), where n is the length of the input iterable (2 in this case). The sort() function has a time complexity of O(n log n), where n is the length of the input list. The space complexity is O(n), as the function creates a list of length n to store the Euclidean distances and tile coordinates.

#### **min_steps(matrix, start, end)**
This function takes a matrix as input, which represents a grid, as well as the starting position and the end position in the grid, and returns the minimum number of steps required to move from the starting position to the end position, if a path exists.

The function starts by performing checks to ensure that the end position and starting position are not blocked by a wall. If either of these positions is blocked by a wall, then the function returns "No path found.".

If there is no wall blocking the start and end position, the function checks if there is a possible movement from the starting position by calling the `movement_check()` function, which returns the available movements that can be made from a given position. If there is no possible movement, the function returns "No path found.".

If there is at least one possible movement, the function proceeds to find the path from the start position to the end position. The `closest_to_endpoint()` function is called to find the next position to move to. The `closest_to_endpoint()` function takes the available movements and returns the next position to move to that is closest to the endpoint. If the next position returned is already in the `previously_visited` list, it will find the next closest position. If there is no other position that can be visited, the function removes the last visited position from the `previously_visited` list, and adds it to the `exile_points` list, and then calls `closest_to_endpoint()` again with the new parameters.

The while loop continues until the current position is the endpoint. Once the loop ends, the function returns the length of the `previously_visited` list minus one, as the length of the `previously_visited` list is equal to the number of steps taken to reach the endpoint.

The time complexity of the `wall_check()` function is O(nm), where n is the number of rows and m is the number of columns of the matrix, as it needs to iterate over each row and each column to check if there are any walls in those locations.

The time complexity of the `movement_check()` function is O(1), as it only needs to check the surrounding tiles.

The time complexity of the `closest_to_endpoint()` function is O(n^2), where n is the number of available movements, as it needs to calculate the distance between each movement and the endpoint.

The time complexity of the `min_steps()` function is O(n^3), as it calls `closest_to_endpoint()` in a while loop, and each call to `closest_to_endpoint()` has a time complexity of O(n^2). The space complexity of the function is O(n), where n is the length of the `previously_visited` list. This is because the function only stores the previously visited positions and the `exile_points` list, which can grow up to the size of the `previously_visited` list, depending on how many times the function needs to remove the last visited position from the `previously_visited` list.